# Module's creation/importation

In [1]:
from tkinter import *


# Considering A, X, B, C, D and every points in inputs as objects defined bellow
def dist(A, X):
    dist = abs(A.x-X.x) + abs(A.y-X.y)
    return dist


def imprecision(A, B):
    if abs(A.x-B.x) <= 5 and abs(A.y-B.y) <= 5:
        return True
    else:
        return False

def determinantOf3points(A, B, C):
    det = A.x*B.y + B.x*C.y + C.x*A.y - B.x*A.y - C.x*B.y - A.x*C.y
    return det


def signDeterminant(A, B, C):
    if determinantOf3points(A, B, C) > 0:
        return("+")
    else:
        return("-")
    
def orientation(G, A, B, C):
    if signDeterminant(G, B, A) == signDeterminant(G, B, C):
        return True
    else:
        return False

    
def barycenter(A, B, C, D):
    if determinantOf3points(A, B, C) + determinantOf3points(B, A, D) != 0:  
        i_x = (determinantOf3points(A, B, C)*D.x + determinantOf3points(B, A, D)*C.x) / (determinantOf3points(A, B, C) + determinantOf3points(B, A, D))
        i_y = (determinantOf3points(A, B, C)*D.y + determinantOf3points(B, A, D)*C.y) / (determinantOf3points(A, B, C) + determinantOf3points(B, A, D))
        i = Point(i_x, i_y, None)
    else:
        # if we have 2 parallel lines
        i = 0
    return i


# Defining our Class

In [2]:
class Point():
    
    def __init__(self, x, y, draw):
        self.x = x
        self.y = y
        self.draw = draw
        
    # is the point on the segment [AB]
    def onSeg(self, A, B):
        if min(A.x, B.x) - 2 <= self.x <= max(B.x, A.x) + 2 and min(A.y, B.y) - 2 <= self.y <= max(B.y, A.y) + 2:
            return True
        else :
            return False
    
    # is the point the intersection between the segment [AB] and the segment [XY]
    def is_IntersectionOfSegs(self, A, B, X, Y):
        if self.onSeg(A, B) == True and self.onSeg(X, Y) == True:
            return True
        else:
            return False
        
    # is the point an intersection between the segment [AB] and the line defined by the segment [XY]
    def is_IntersectionOfSeg_Line(self, A, B, X, Y):
        if self.onSeg(A, B) == True and self.onSeg(B, X) == False:
            return True
        else:
            return False
    
    # is the point an intersection between the segment [AB] and the ray [XY[ staring from the point X
    def is_IntersectionSeg_Ray(self, A, B, X, Y):
        if self.onSeg(A, B) == True and self.onSeg(X, Y) == False:
            if dist(X, self) > dist(Y, self):
                return True
            else :
                return False
    
    # is this point visible or is there any segment == wall hiding the vision of the point
    def notVisible(self, A, B, c, g):
        if self.onSeg(A, B) == True and self.onSeg(c, g) == True:
            return True
        else:
            return False
     
    # we can see this point, but can we see further ==> this case bellow :
    #  \--•B
    #   \
    #    •A        We can see the point A, but we can also see the point B behind from G   
    #   /
    #  /
    #   •G
    def ableToSeeFurther(self, A, B, c, g):
        if self.onSeg(A, B) == True and c.onSeg(self, g) == True:
            return True
        else:
            return False

# Building the room

In [3]:
corners = []

# Link the click with the canva to create the map
def create_map():
    cnv.bind(("<Button-3>"), validate_point)

# Verify each click to avoid mistakes by creating the map
def validate_point(event):
    global corners_list
    global room
    x, y = event.x, event.y
    point = Point(x, y, None)
    corners.append(point)
    size = len(corners)
    text.set('Place every corners of the room by right clicking.')
    
    # if this is the first point we place, just show it by drawing a blue circle
    if size == 1:
        point.draw = cnv.create_oval(event.x-3, event.y-3, event.x+3, event.y+3, fill='#00E676')
      
    # if we have 2 points or more, we want to enable user to finish the room by clicking on an existing point
    if size >= 2: 
        
        if imprecision(corners[size-1], corners[0]) == True:
            cnv.create_line(corners[size-2].x, corners[size-2].y, corners[0].x, corners[0].y, fill='#00E676', tag='border')
            
            corners_list = []
            for objects in corners:
                corners_list.append((objects.x, objects.y))

            del corners_list[-1]
            room = cnv.create_polygon(corners_list, fill='#00E676', tag='poly')
            cnv.dtag('border')
            b_g['state'] = NORMAL
            b_m['state'] = DISABLED
            cnv.unbind("<Button-3>")
            text.set('Click on "Place Guard" to place the guard inside the room.')
        
        # if we don't have more than 3 points, we can just draw blue circles and link the one we placed with the previous one
        elif size == 2 or size == 3:
            point.draw = cnv.create_oval(event.x-3, event.y-3, event.x+3, event.y+3, fill='#00E676')
            cnv.create_line(corners[size-2].x, corners[size-2].y, corners[size-1].x, corners[size-1].y, fill='#00E676', tag='border')

        
        # when we have more than 3 points, we could place a point that makes the segment crossing another one, we don't want that happens
        elif size > 3:
                c = 0
                for n in range(size-2):
                    i = barycenter(point, corners[size-2], corners[n], corners[n+1])
                    if i.is_IntersectionOfSegs(point, corners[size-2], corners[n], corners[n+1]) == True and imprecision(i, corners[size-2]) == False:
                        c += 1

                        

                if c != 0:
                    del corners[-1]
                    text.set("You can't place this corners here !")
                else:
                    point.draw = cnv.create_oval(point.x-3, point.y-3, point.x+3, point.y+3, fill='#00E676')
                    cnv.create_line(corners[size-2].x, corners[size-2].y, corners[size-1].x, corners[size-1].y, fill='#00E676', tag='border')


def motionMap():
    cnv.bind("<B1-Motion>", validateNewPosition)

def validateNewPosition(event):
    x, y = event.x, event.y
    global room
    CLICK_DETECTION = True
    cnv.delete(room)
    

    for n, p in enumerate(corners_list):
        if p[0]-10 <= x <= p[0]+10 and p[1]-10 <= y <= p[1]+10 and CLICK_DETECTION == True:
            cnv.coords(corners[n].draw, x-10, y-10, x+10, y+10)
            corners_list[n] = (x, y)
            room = cnv.create_polygon(corners_list, fill='#00E676', tag='poly')
            corners[n].x, corners[n].y = x, y
            if placed == True:
                visualisation(guard, corners)
            else:
                pass

CLICK_DETECTION = False

# Placing the guard

In [4]:
def guard():
    cnv.bind("<Button-1>", validate_guard)
    
    
# is he inside the room ? let's do some maths using ray, intersections, barycenters, ...
def validate_guard(event):
    global iconGuard
    global guard
    global placed
    x, y = event.x, event.y
    guard = Point(x, y, None)
    placed = False
    
    # we add the size of the window ==> that makes a ray starting from the click we made going to the same high point W pixels further
    ray = Point(event.x + W, event.y, None)
    
    # and we count how many intersections we have / peer ==> outside : because I enter and I leave the room
    nb_inter = 0
    
    for p in range(len(corners)-1):
        i = barycenter(guard, ray, corners[p], corners[p+1])

        if i != 0:
            if i.onSeg(corners[p], corners[p+1]) == True and i.onSeg(guard, ray) == True and i != 0:
                nb_inter += 1
        
            
    if placed == False and nb_inter % 2 == 1:
        placed = True
        iconGuard = cnv.create_oval(event.x - 10, event.y - 10, event.x + 10, event.y  + 10, fill='#FF3D00', tag='guard')
        cnv.tag_raise('guard')
        text.set('You can place another guard if you want')
        
        cnv.unbind("<Button-1>")
        visualisation(guard, corners)
    
    elif nb_inter % 2 == 0:
        text.set('You have to place the guard inside the room')
        
    else:
        text.set('Click again on "Place a Guard" to place another guard')
        
        
    cnv.unbind("<Button-3>")

def drag():
    cnv.bind("<B1-Motion>", motionGuard)

def motionGuard(event):
    cnv.delete(vision)
    x, y = event.x, event.y
    ray = Point(x + W, y, None)
    newPosition = Point(x, y, None)
    CLICK_DETECTION = True

    if guard.x-10 <= x <= guard.x+10 and guard.y-10 <= y <= guard.y+10 and CLICK_DETECTION == True:
        guard.x, guard.y = x, y
        nb_inter = 0
        for p in range(len(corners)-1):
            i = barycenter(guard, ray, corners[p], corners[p+1])
            if i != 0:
                if i.onSeg(corners[p], corners[p+1]) == True and i.onSeg(guard, ray) == True and i != 0:
                    nb_inter += 1

        if nb_inter % 2 == 1:
            cnv.coords(iconGuard, x-10, y-10, x+10, y+10)
            visualisation(guard, corners)

        else:
            text.set("Stay inside the room")
            
        
        
        
CLICK_DETECTION = False

# Optimal Place / Area - Surface

In [6]:
# here is another hard problem to solve about how to determine the area/surface of any polygon not only basics one

def optimization():
    pass

# Main

In [7]:
def visualisation(guard, corners):

    global pointsSeen
    global pointsNotSeen
    
    pointsSeen = []
    pointsNotSeen = []
    
    corners[-1] = corners[0]
    
    for seg in range(len(corners) - 1):
        for point in range(len(corners)):
            i = barycenter(corners[seg], corners[seg+1], corners[point], guard)
            
            # have to consider the case where there is no interesection (parallelism)
            if i == 0:
                pass
            
            elif i.notVisible(corners[seg], corners[seg+1], corners[point], guard) == True and imprecision(i, corners[point]) == False:
                pointsNotSeen.append(corners[point])
                
    # managing 2 lists to keep in memory corners we can't reach
    pointsNotSeen = [p for p in corners if p in pointsNotSeen]
    pointsSeen = [p for p in corners if p not in pointsNotSeen]
    
    vision = []
    for n in range(len(pointsSeen)):
        vision.append((pointsSeen[n].x, pointsSeen[n].y))

    
    # the fact of having made this loop above reduces drastically the calculation we will do here 
    for point in range(len(pointsSeen)):
        alignedPoints = []
        for seg in range(len(corners)-1):
            
            i = barycenter(corners[seg], corners[seg+1], pointsSeen[point], guard)
            if i.ableToSeeFurther(corners[seg], corners[seg+1], pointsSeen[point], guard) == True and imprecision(i, pointsSeen[point]) == False:
                for n, pt in enumerate(corners):
                    if pointsSeen[point] == pt:
                        
                        # the sign of the determinant enables us to know the orientation/order ==> ABC or ACB or BAC ? 
                        # A --- B --- C // A --- C --- B // B --- A --- C  so we use matrices, determinants to understand geometry of matrices
                        if orientation(guard, corners[n-1], pointsSeen[point], corners[n+1]) == True:
                            alignedPoints.append(i)
                                
        alignedPoints.sort(key=lambda P:dist(P, guard))
        
        # managing the case where we are able to see further so when we have values in sides

        # h----•C-g
        #         \  
        # e\--•B---\f
        #   \
        #    •A        We can see the point A, but we can also see the point B behind from G, but the point C is hidden.   
        #   /          So we sorted alignedPoints list considering the distance from the guard and now we work on !
        #  /
        #   •G
        
        # The question is : we have a list with every points sorted in the right order but when we add the point B,
        # we have to put it between the 2 points of the segment where it is. 
        if len(alignedPoints) != 0:
            for p in range(len(corners)-1):
                if pointsSeen[point] == corners[p]:
                    for p2 in range(len(corners)-1):
                        
                        if alignedPoints[0].onSeg(corners[p2], corners[p2 + 1]) == True and p < p2:
                            pointsSeen.insert(point+1, alignedPoints[0])
                            point += 1
                            
                        elif alignedPoints[0].onSeg(corners[p2], corners[p2 + 1]) == True and p > p2:
                            pointsSeen.insert(point, alignedPoints[0])
                            point += 1
                            
    draw(pointsSeen, guard)

# Visualisation

In [8]:
def draw(pointsSeen, guard):
    global vision
    list_points = []
    for point in pointsSeen:
        list_points.append((point.x, point.y))

    vision = cnv.create_polygon(list_points, fill='#FFEA00', tag='vision')
    cnv.tag_raise('guard')
    cnv.create_oval(guard.x - 10, guard.y - 10, guard.x + 10, guard.y  + 10, fill='#FF3D00')


    return

# Building the UI

In [9]:
H, W = 500, 750

win = Tk()
win.title("Visualisation")
cnv = Canvas(win, width = W, height = H, bg='white')
cnv.pack(side=TOP)

text = StringVar()
text.set('Click on "Create map" to place every corners one by one')
txt = Label(cnv, height=2, textvariable=text, bg='#00B0FF', font=('Helvetica', 10), justify='center')
txt.place(x=2, y=2, height=50, width=750)

frm2 = Frame(win, bg='#00B0FF', padx=232, relief='flat', bd=10)
frm2.pack(side=BOTTOM)
b_d = Button(frm2, text='Drag Guard', command=drag)
b_d.pack(side=RIGHT)
b_m = Button(frm2, text='Create map', command=create_map)
b_m.pack(side=LEFT)
b_g = Button(frm2, text='Place guard', command=guard)
b_g['state'] = DISABLED
b_g.pack(side=LEFT)
b_o = Button(frm2, text='Change Shape', command=motionMap)
b_o.pack(side=RIGHT)
cnv.unbind("<Button-1>")
